In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, dot
from tensorflow.keras import regularizers
from sklearn.decomposition import TruncatedSVD
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from tqdm import trange, notebook

In [4]:
user_inventory=pd.read_csv("./Data/embeding_rating.csv")
game_info=pd.read_csv("./data/game_detail_preprocessing.csv")

In [5]:
del user_inventory["Unnamed: 0"]
del game_info["Unnamed: 0"]

In [6]:
user_inventory.head(3)

,Counter-Strike: Global Offensive,Grand Theft Auto V,Total War: SHOGUN 2,Path of Exile,Life is Feudal: Your Own,Call of Duty: Modern Warfare 2 - Multiplayer,Chivalry: Medieval Warfare,Call of Duty: Black Ops II,Infestation: Survivor Stories 2020,Space Engineers,...,Sakura Space,Negligee,Escape Dead Island,The Forgotten Ones,F1 2014,Wild Animal Racing,Luna Online: Reborn,Shantae and the Seven Sirens,Epic Battle Fantasy 4,Danganronpa 2: Goodbye Despair
0,0.460547,0.257275,0.107505,0.640422,0.340797,0.449093,0.043132,0.037675,0.000306,0.041271,...,0.657018,0.088584,0.150951,0.535680,0.162711,0.215170,0.376517,0.469047,0.091355,0.104336
1,1.236492,0.947491,0.336702,3.873230,0.781555,0.982932,0.098145,0.077848,0.002558,0.108208,...,1.339629,0.245297,0.378320,1.130005,0.372688,0.620238,1.274697,0.970412,0.213148,0.291913
2,1.227606,0.965474,0.318682,2.669402,0.785304,0.982270,0.103198,0.075032,0.002837,0.103871,...,1.319382,0.247969,0.376239,1.189568,0.376436,0.591505,1.186636,1.059214,0.219903,0.314533


In [7]:
game_info.head(3)

,game name,description,price,categories,genres,recommendations
0,Counter-Strike,Play the world's number 1 online action game. ...,"₩ 10,500","['Multi-player', 'PvP', 'Online PvP', 'Shared/...",Action,98065.0
1,Team Fortress Classic,One of the most popular online action games of...,"₩ 5,500","['Multi-player', 'PvP', 'Online PvP', 'Shared/...",Action,3664.0
2,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,"₩ 5,500","['Multi-player', 'PvP', 'Online PvP', 'Shared/...",Action,1250.0


In [8]:
def Collaborative_Filtering(User_Inventory,Base_Game):
    matrix=User_Inventory.values.T
    corr=np.corrcoef(matrix)
    game_name=User_Inventory.columns
    game_name_list=list(game_name)
    input_value=game_name_list.index(Base_Game)
    corr_input_value=corr[input_value]
    Recommend_result=list(game_name[corr_input_value>=0.95])[:20]
    for recommend_game in Recommend_result:
        if recommend_game== Base_Game:
            Recommend_result.remove(recommend_game)
    return Recommend_result

In [9]:
def content_based_filtering(GameData,Input_game):
    
    dic_recommend={}
    app_name=GameData["game name"]
    tfidf_vectorizer = TfidfVectorizer(analyzer="word",strip_accents = 'unicode',stop_words='english')
    tfidf = tfidf_vectorizer.fit_transform(list(GameData['description']))
    for index in range(GameData['description'].shape[0]):
        cosine_similar = linear_kernel(tfidf[index],tfidf).flatten()
        related_rating = (-cosine_similar).argsort()[:21]   
        dic_recommend.update({app_name[index]:[app_name[i] for i in related_rating]})
    content_based_results = pd.DataFrame(dic_recommend)
    content_based_results.reset_index(inplace=True)
    content_based_results = content_based_results.T
    result=content_based_results.drop(0,axis=1)
    
    return list(result.loc[Input_game])

In [10]:
Collaborative_Filtering(user_inventory,"Counter-Strike")

['Counter-Strike: Global Offensive',
 'Grand Theft Auto V',
 'Total War: SHOGUN 2',
 'Path of Exile',
 'Life is Feudal: Your Own',
 'Call of Duty: Modern Warfare 2 - Multiplayer',
 'Chivalry: Medieval Warfare',
 'Space Engineers',
 '7 Days to Die',
 'Dream Car Builder',
 'Godot Engine',
 'Trailmakers',
 'Kerbal Space Program',
 'Mount & Blade: Warband',
 'Trine',
 'The Showdown Effect',
 'Monaco',
 'Killing Floor',
 'Shadowrun Returns',
 'Red Faction: Armageddon']

In [11]:
content_based_filtering(game_info,"Counter-Strike")

['Team Fortress Classic',
 'Insurgency',
 "Tom Clancy's Rainbow Six® 3 Gold",
 'Call of Duty® 4: Modern Warfare®',
 'F.E.A.R.',
 'Lead and Gold: Gangs of the Wild West',
 'Heroes of Might & Magic V',
 'Arma Tactics',
 'Super Star',
 'Call of Duty®: Modern Warfare® 2',
 'Takedown: Red Sabre',
 'Call of Duty® 2',
 'Jagged Alliance 2 - Wildfire',
 'Silent Storm Gold Edition',
 'Ricochet',
 'Call of Duty: World at War',
 'Phasmophobia',
 'The Room',
 'Head Shot',
 'Interstellar Marines']

In [34]:
def Hybrid_Filtering(User_Inventory,GameData,Base_Game):
    Col_GameList=Collaborative_Filtering(User_Inventory,Base_Game)
    Con_GameList=content_based_filtering(GameData,Base_Game)
    result=[]
    Col_recom_index=[]
    Con_recom_index=[]
    cnt=0
    finish=False
    
    while(finish):
        for i in notebook.tqdm(range(len(Col_GameList))):
            for j in range(len(Con_GameList)):
                if Col_GameList[i]==Con_GameList[j]:
                    result.append(Con_GameList[j])
                    Col_recom_index.append(i)
                    Con_recom_index.append(j)
                    cnt+=1
        finsih=True
    
    if cnt<10:
        while(cnt<10):
            for i in range(10):
                if i not in Col_recom_index:
                    result.append(Col_GameList[i])
                    cnt+=1
                if cnt<10:
                    result.append(Con_GameList[i])
                    cnt+=1
                else:
                    break
        
    print("<Recommended Game List>\n")
    result=result[0:10]
    for i in range(len(result)):
        print(i+1,"위\t",result[i],end="\n")

In [35]:
Hybrid_Filtering(user_inventory,game_info,"Counter-Strike")

<Recommended Game List>

1 위	 Counter-Strike: Global Offensive
2 위	 Team Fortress Classic
3 위	 Grand Theft Auto V
4 위	 Insurgency
5 위	 Total War: SHOGUN 2
6 위	 Tom Clancy's Rainbow Six® 3 Gold
7 위	 Path of Exile
8 위	 Call of Duty® 4: Modern Warfare®
9 위	 Life is Feudal: Your Own
10 위	 F.E.A.R.
